In [195]:
import sqlite3
from pyhwpx import Hwp
import os

#region 공통 데이터
파일이름 = ""
현장명 = ""
현장소재지 = ""
공사금액 = 0.0
보고서_날짜_년 = ""
보고서_날짜_월 = ""
시공자 = ""
설계자 = ""
공사개요_대상공사 = ""
공사개요_구조 = ""
공사개요_개소 = 0
공사개요_층수지하 = 0
공사개요_층수지상 = 0
공사개요_굴착깊이 = 0.0
공사개요_최고높이 = 0.0
공사개요_연면적 = 0.0
기타특수구조물개요 = ""
주요공법1 = ""
주요공법2 = ""
주요공법3 = ""
주요공법4 = ""
주요공법5 = ""
주요공법6 = ""
주요공법7 = ""
주요공법8 = ""
주요공법9 = ""
주요공법10 = ""
파일경로 = ""
비고 = ""
cursor_positions = []  # 문단 앞 커서 위치를 담는 리스트 (list, para, pos)
section_titles = []    # 각 위치에 대응하는 목차 또는 섹션 이름 **목차명=필드명이되도록할것
custom_data = []

def 데이터_초기화():
    global 파일이름
    global 현장명 
    global 현장소재지 
    global 공사금액 
    global 보고서_날짜_년 
    global 보고서_날짜_월 
    global 시공자 
    global 설계자 
    global 공사개요_대상공사
    global 공사개요_구조 
    global 공사개요_개소 
    global 공사개요_층수지하 
    global 공사개요_층수지상 
    global 공사개요_굴착깊이 
    global 공사개요_최고높이 
    global 공사개요_연면적 
    global 기타특수구조물개요
    global 주요공법1 
    global 주요공법2 
    global 주요공법3 
    global 주요공법4 
    global 주요공법5 
    global 주요공법6 
    global 주요공법7 
    global 주요공법8 
    global 주요공법9 
    global 주요공법10
    global 파일경로 
    global 비고 
    global cursor_positions
    global section_titles
    global custom_data 
    파일이름 = ""
    현장명 = ""
    현장소재지 = ""
    공사금액 = 0.0
    보고서_날짜_년 = ""
    보고서_날짜_월 = ""
    시공자 = ""
    설계자 = ""
    공사개요_대상공사 = ""
    공사개요_구조 = ""
    공사개요_개소 = 0
    공사개요_층수지하 = 0
    공사개요_층수지상 = 0
    공사개요_굴착깊이 = 0.0
    공사개요_최고높이 = 0.0
    공사개요_연면적 = 0.0
    기타특수구조물개요 = ""
    주요공법1 = ""
    주요공법2 = ""
    주요공법3 = ""
    주요공법4 = ""
    주요공법5 = ""
    주요공법6 = ""
    주요공법7 = ""
    주요공법8 = ""
    주요공법9 = ""
    주요공법10 = ""
    파일경로 = ""
    비고 = ""
    cursor_positions = []  # 문단 앞 커서 위치를 담는 리스트 (list, para, pos)
    section_titles = []    # 각 위치에 대응하는 목차 또는 섹션 이름 **목차명=필드명이되도록할것
    custom_data = []

#endregion

#region 데이터베이스 초기화 및 연결 함수
def connect_db():
    conn = sqlite3.connect('data.db')
    cursor = conn.cursor()
    return conn, cursor

# 테이블 생성 함수
def initialize_db():
    conn, cursor = connect_db()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS file_data (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            파일이름 TEXT,
            현장명 TEXT,
            현장소재지 TEXT,
            공사금액 REAL,
            보고서_날짜_년 TEXT,
            보고서_날짜_월 TEXT,
            시공자 TEXT,
            설계자 TEXT,
            공사개요_대상공사 TEXT,
            공사개요_구조 TEXT,
            공사개요_개소 INTEGER,
            공사개요_층수지하 INTEGER,
            공사개요_층수지상 INTEGER,
            공사개요_굴착깊이 REAL,
            공사개요_최고높이 REAL,
            공사개요_연면적 REAL,
            기타특수구조물개요 TEXT,
            주요공법1 TEXT,
            주요공법2 TEXT,
            주요공법3 TEXT,
            주요공법4 TEXT,
            주요공법5 TEXT,
            주요공법6 TEXT,
            주요공법7 TEXT,
            주요공법8 TEXT,
            주요공법9 TEXT,
            주요공법10 TEXT,
            파일경로 TEXT,
            섹션명 TEXT,
            리스트 INTEGER,
            para INTEGER,
            pos INTEGER,
            비고 TEXT
        )
    ''')
    conn.commit()
    conn.close()

# 데이터 삽입 함수 / 문서 전체를 한방에 db화함.
def insert_data(
    파일이름, 현장명, 현장소재지, 공사금액, 보고서_날짜_년, 보고서_날짜_월,
    시공자, 설계자, 공사개요_대상공사, 공사개요_구조, 공사개요_개소,
    공사개요_층수지하, 공사개요_층수지상, 공사개요_굴착깊이,
    공사개요_최고높이, 공사개요_연면적, 기타특수구조물개요,
    주요공법1, 주요공법2, 주요공법3, 주요공법4, 주요공법5,
    주요공법6, 주요공법7, 주요공법8, 주요공법9, 주요공법10,
    파일경로, 섹션명_리스트, 데이터_리스트, 비고,
): 
    """
    파일이름~파일경로: 공통 값으로 한 번만 입력.
    섹션명_리스트: 섹션명을 담은 리스트.
    데이터_리스트: (리스트, para, pos) 튜플 리스트.
    """
    if len(섹션명_리스트) != len(데이터_리스트):
        raise ValueError("섹션명_리스트와 데이터_리스트의 길이가 다릅니다.")
    
    conn, cursor = connect_db()
    
    # 각 섹션 데이터 삽입
    for 섹션명, (리스트, para, pos) in zip(섹션명_리스트, 데이터_리스트):
        cursor.execute('''
            INSERT INTO file_data (
                파일이름, 현장명, 현장소재지, 공사금액, 보고서_날짜_년, 보고서_날짜_월,
                시공자, 설계자, 공사개요_대상공사, 공사개요_구조, 공사개요_개소,
                공사개요_층수지하, 공사개요_층수지상, 공사개요_굴착깊이,
                공사개요_최고높이, 공사개요_연면적, 기타특수구조물개요,
                주요공법1, 주요공법2, 주요공법3, 주요공법4, 주요공법5,
                주요공법6, 주요공법7, 주요공법8, 주요공법9, 주요공법10,
                파일경로, 섹션명, 리스트, para, pos, 비고
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            파일이름, 현장명, 현장소재지, 공사금액, 보고서_날짜_년, 보고서_날짜_월,
            시공자, 설계자, 공사개요_대상공사, 공사개요_구조, 공사개요_개소,
            공사개요_층수지하, 공사개요_층수지상, 공사개요_굴착깊이,
            공사개요_최고높이, 공사개요_연면적, 기타특수구조물개요,
            주요공법1, 주요공법2, 주요공법3, 주요공법4, 주요공법5,
            주요공법6, 주요공법7, 주요공법8, 주요공법9, 주요공법10,
            파일경로, 섹션명, 리스트, para, pos, 비고
        ))

    conn.commit()
    conn.close()

# 데이터 조회 함수 # todo 
def fetch_data():
    conn, cursor = connect_db()
    cursor.execute('SELECT * FROM file_data')
    rows = cursor.fetchall()
    conn.close()
    return rows

#데이터 조회2 ##이건 좀더 고민해야함 todo
def fetch_data_with_fallback(column=None, keyword=None):
    """특정 열 검색 / 모든 열 검색 / 조건 없이 조회"""
    conn, cursor = connect_db()
    if column and keyword:  # 특정 열에 대한 검색
        query = f'SELECT * FROM file_data WHERE {column} LIKE ?'
        cursor.execute(query, (f'%{keyword}%',))
    elif keyword:  # 모든 열에서 검색
        query = '''
            SELECT * FROM file_data
            WHERE 파일이름 LIKE ? OR 현장명 LIKE ? OR 현장소재지 LIKE ? OR 공사개요_대상공사 LIKE ?
        '''
        cursor.execute(query, (f'%{keyword}%',) * 4)
    else:  # 조건 없이 전체 조회
        query = 'SELECT * FROM file_data'
        cursor.execute(query)

    rows = cursor.fetchall()
    conn.close()
    return rows

#custom_data로 추출하기
def fetch_custom_data():
    """
    데이터베이스에서 데이터를 가져와 custom_data 형식으로 반환합니다.
    """
    conn, cursor = connect_db()
    
    # 필요한 데이터만 SELECT (리스트, para, pos와 섹션명만 추출)
    cursor.execute('SELECT 리스트, para, pos, 섹션명 FROM file_data ORDER BY id')
    rows = cursor.fetchall()  # 데이터베이스에서 행 가져오기
    conn.close()
    
    # 데이터를 custom_data 형식으로 변환
    custom_data = [((row[0], row[1], row[2]), row[3]) for row in rows]
    return custom_data

#custom_data로 특정id만 추출하기(id의 위치),(id+1의 위치)(id의 섹션명)
def fetch_custom_data_with_next_positions(id_list):
    """
    주어진 ID 리스트에 해당하는 데이터를 custom_data 형식으로 반환.
    (id의 시작 좌표), (id+1의 시작 좌표), "id 섹션명" 구조로 변환.
    """
    conn, cursor = connect_db()
    
    # ID 리스트를 조건으로 데이터 가져오기
    placeholders = ', '.join('?' for _ in id_list)
    query = f'''
        SELECT id, 리스트, para, pos, 섹션명 
        FROM file_data 
        WHERE id IN ({placeholders}) 
        ORDER BY id
    '''
    cursor.execute(query, id_list)
    rows = cursor.fetchall()
 

     # ID의 다음 위치를 가져오기 위한 사전 조회
    next_positions = {}
    for id_ in id_list:
        cursor.execute('''
            SELECT 리스트, para, pos 
            FROM file_data 
            WHERE id = ?
        ''', (id_ + 1,))
        next_row = cursor.fetchone()
        if next_row:
            next_positions[id_] = next_row
        else:
            next_positions[id_] = "문서끝"

    # 데이터 변환
    custom_data = []
    for row in rows:
        current_id, 리스트, para, pos, 섹션명 = row
        next_position = next_positions.get(current_id, "문서끝")
        if next_position != "문서끝":
            custom_data.append(
                ((리스트, para, pos), (next_position[0], next_position[1], next_position[2]), 섹션명)
            )
        else:
            custom_data.append(
                ((리스트, para, pos), "문서끝", 섹션명)
            )
    conn.close()
    return custom_data

# 데이터 삭제 함수
def delete_data(row_id):
    conn, cursor = connect_db()
    cursor.execute('DELETE FROM file_data WHERE id = ?', (row_id,))
    conn.commit()
    conn.close()

# 데이터 업데이트 함수  ##이것도 잘모르겠음. todo
def update_data(row_id, **kwargs):
    """
    특정 ID의 데이터를 업데이트합니다.
    kwargs: 업데이트할 열 이름과 값의 딕셔너리 (예: {"파일이름": "새 파일명", "현장명": "새 현장명"})
    """
    conn, cursor = connect_db()
    updates = []
    values = []

    for key, value in kwargs.items():
        updates.append(f"{key} = ?")
        values.append(value)
    
    values.append(row_id)
    query = f'UPDATE file_data SET {", ".join(updates)} WHERE id = ?'
    cursor.execute(query, values)
    conn.commit()
    conn.close()

# 데이터 출력 함수
def print_data():
    rows = fetch_data()
    for row in rows:
        print(row)

def reset_table():
    """
    테이블을 삭제하고 다시 생성하여 ID를 포함한 모든 데이터를 초기화합니다.
    """
    conn, cursor = connect_db()

    # 테이블 삭제
    cursor.execute('DROP TABLE IF EXISTS file_data')
    conn.commit()

    # 테이블 재생성
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS file_data (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            파일이름 TEXT,
            현장명 TEXT,
            현장소재지 TEXT,
            공사금액 REAL,
            보고서_날짜_년 TEXT,
            보고서_날짜_월 TEXT,
            시공자 TEXT,
            설계자 TEXT,
            공사개요_대상공사 TEXT,
            공사개요_구조 TEXT,
            공사개요_개소 INTEGER,
            공사개요_층수지하 INTEGER,
            공사개요_층수지상 INTEGER,
            공사개요_굴착깊이 REAL,
            공사개요_최고높이 REAL,
            공사개요_연면적 REAL,
            기타특수구조물개요 TEXT,
            주요공법1 TEXT,
            주요공법2 TEXT,
            주요공법3 TEXT,
            주요공법4 TEXT,
            주요공법5 TEXT,
            주요공법6 TEXT,
            주요공법7 TEXT,
            주요공법8 TEXT,
            주요공법9 TEXT,
            주요공법10 TEXT,
            파일경로 TEXT,
            섹션명 TEXT,
            리스트 INTEGER,
            para INTEGER,
            pos INTEGER,
            비고 TEXT
        )
    ''')
    conn.commit()
    conn.close()
    print("테이블이 초기화되었습니다.")
    
# 데이터베이스 초기화
initialize_db()

#endregion

#region hwp 초기화

hwp = Hwp(new=True)
#hwp2 = Hwp(new=True)

#hwp.Run("FileOpen")
#복붙이 되는지 확인
hwp.Open(r"C:\Users\codek\Desktop\건설자료\01. [변경] 안전관리계획서 제1편 - 천안 성성5지구 IPARK 신축공사(R4) 정기안전점검 수정 240925.hwp")
#hwp2.Open(r"C:\Users\codek\Documents\GitHub\connwnw\test\샘플_템플릿.hwp")

hwp.MoveDocBegin()

#캐럿 포지션 저장
def 위치추가():
    #목차 문단 맨 앞의 위치 추가
    cursor_positions.append(hwp.GetPos())

    #목차 문단 셀선택 후 추가하고 다시 첫 위치로 돌아가기
    hwp.MoveSelParaEnd()
    section_titles.append(hwp.get_selected_text())
    hwp.MoveParaBegin()
#마지막 처음 추가하기
def 처음지점추가():
    hwp.MoveDocBegin()
    cursor_positions.append(hwp.GetPos())
    section_titles.append("문서처음")
def 문단시작지점추가():
        문단시작_list =[]
        for i in hwp.ctrl_list:
            if i.UserDesc == "새 번호":#컨트롤이 표일 경우
                문단시작_list.append(i)#리스트에 저장
        for i in range(len(문단시작_list)):
            hwp.move_to_ctrl(문단시작_list[i])
            hwp.MoveLeft()
            cursor_positions.append(hwp.GetPos())
            section_titles.append(f"문단{i+1}")
#마지막 지점 추가하기
def 마지막지점추가():
    hwp.MoveDocEnd()
    cursor_positions.append(hwp.GetPos())
    section_titles.append("문서끝")
#리스트 para기준 재정렬
def 재정렬_파라기준(cursor_positions, section_titles):
    """
    cursor_positions의 'para' 값을 기준으로 cursor_positions와 section_titles를 재정렬하는 함수.
    
    매개변수:
    - cursor_positions (list of tuples): (table, para, dotPos) 형식의 커서 위치 리스트.
    - section_titles (list of str): 해당 커서 위치에 대응하는 섹션 제목 리스트.
    
    반환값:
    - tuple: 정렬된 cursor_positions와 section_titles.
    """
    # cursor_positions와 section_titles를 결합
    결합_데이터 = list(zip(cursor_positions, section_titles))
    
    # 'para' 값(튜플의 두 번째 값)을 기준으로 정렬
    정렬된_데이터 = sorted(결합_데이터, key=lambda x: x[0][1])
    
    # 정렬된 데이터를 다시 두 개의 리스트로 분리
    정렬된_cursor_positions, 정렬된_section_titles = zip(*정렬된_데이터)
    
    return list(정렬된_cursor_positions), list(정렬된_section_titles)

#endregion

#################################################################

#region 문서 공통 info 추출
#파일경로 추출
파일경로 = hwp.Path

#파일이름 추출
파일이름 = os.path.basename(파일경로)
print(파일경로, 파일이름)



#endregion


C:\Users\codek\Desktop\건설자료\01. [변경] 안전관리계획서 제1편 - 천안 성성5지구 IPARK 신축공사(R4) 정기안전점검 수정 240925.hwp 01. [변경] 안전관리계획서 제1편 - 천안 성성5지구 IPARK 신축공사(R4) 정기안전점검 수정 240925.hwp


In [ ]:
###--공사개요서 뒤지기--###
def 오른_표이동(n):
    """n번 오른쪽으로 이동"""
    for _ in range(n):
        hwp.TableRightCell()

##처음 위치로##
hwp.Cancel()
hwp.MoveDocBegin()

if hwp.find_forward("1. 공사 개요서"):
    hwp.SelectCtrlFront()
    hwp.ShapeObjTableSelCell() # 첫번째 셀 선택
else : print("공사개요서 못찾음.")

#현장명
오른_표이동(2)
print(hwp.get_selected_text())
현장명 = "현장명"
현장소재지 = "현장소재지"
공사금액 = 1.0
보고서_날짜_년 = "2025"
보고서_날짜_월 = "09"
시공자 = "시공자"
설계자 = "설계자"
공사개요_대상공사 = "대상공사"
공사개요_구조 = "구조"
공사개요_개소 = 2
공사개요_층수지하 = 3
공사개요_층수지상 = 4
공사개요_굴착깊이 = 5.0
공사개요_최고높이 = 6.7
공사개요_연면적 = 8.9
기타특수구조물개요 = "특수구조물"
주요공법1 = "공법1"
주요공법2 = "공법2"
주요공법3 = "공법3"
주요공법4 = "공법4"
주요공법5 = "공법5"
주요공법6 = "공법6"
주요공법7 = "공법7"
주요공법8 = "공법8"
주요공법9 = "공법9"
주요공법10 = "공법10"
비고 = "비고"

공사개요서 못찾음.
2024. 10 ~ 2027. 12


In [187]:
reset_table()

테이블이 초기화되었습니다.


In [188]:
cursor_positions.clear()
section_titles.clear()

In [185]:
#처음위치추가
처음지점추가()
print(cursor_positions, section_titles)

마지막지점추가()
print(cursor_positions, section_titles)

문단시작지점추가()
print(cursor_positions, section_titles)

[(0, 64, 9), (0, 68, 12), (0, 72, 12), (0, 83, 24), (0, 87, 9), (0, 0, 16)] ['1. 공사 개요서', '2-1. 현장 위치도', '1) 현장 주변현황', '5-1. 가설구조물 등의 설치 및 해체계획', '1) 공통공사', '문서처음']
[(0, 64, 9), (0, 68, 12), (0, 72, 12), (0, 83, 24), (0, 87, 9), (0, 0, 16), (0, 1067, 25)] ['1. 공사 개요서', '2-1. 현장 위치도', '1) 현장 주변현황', '5-1. 가설구조물 등의 설치 및 해체계획', '1) 공통공사', '문서처음', '문서끝']
[(0, 64, 9), (0, 68, 12), (0, 72, 12), (0, 83, 24), (0, 87, 9), (0, 0, 16), (0, 1067, 25), (0, 54, 8), (0, 99, 0), (0, 584, 2), (0, 908, 0)] ['1. 공사 개요서', '2-1. 현장 위치도', '1) 현장 주변현황', '5-1. 가설구조물 등의 설치 및 해체계획', '1) 공통공사', '문서처음', '문서끝', '문단1', '문단2', '문단3', '문단4']


In [ ]:
#문서의 내용 리스트에 추가
섹션리스트=["1. 공사 개요서", "2-1. 현장 위치도", "1) 현장 주변현황", "5-1. 가설구조물 등의 설치 및 해체계획", "1) 공통공사"]
#findforward로 섹션리스트를 순회하면서 위치 추가
def 중간위치추가(sec_list):
    hwp.MoveDocBegin()
    for i in sec_list:
        print(i)
        if (hwp.find_forward(i)):
            위치추가()
        else : print(f"섹션{i}를 찾을수 없음.")
중간위치추가(섹션리스트)
print(cursor_positions, section_titles)

1. 공사 개요서
2-1. 현장 위치도
1) 현장 주변현황
5-1. 가설구조물 등의 설치 및 해체계획
1) 공통공사
[(0, 64, 9), (0, 68, 12), (0, 72, 12), (0, 83, 24), (0, 87, 9)] ['1. 공사 개요서', '2-1. 현장 위치도', '1) 현장 주변현황', '5-1. 가설구조물 등의 설치 및 해체계획', '1) 공통공사']


In [186]:
#para 크기 순으로 재정렬
정렬된_cursor_positions, 정렬된_section_titles = 재정렬_파라기준(cursor_positions, section_titles)
print(정렬된_cursor_positions, 정렬된_section_titles)

[(0, 0, 16), (0, 54, 8), (0, 64, 9), (0, 68, 12), (0, 72, 12), (0, 83, 24), (0, 87, 9), (0, 99, 0), (0, 584, 2), (0, 908, 0), (0, 1067, 25)] ['문서처음', '문단1', '1. 공사 개요서', '2-1. 현장 위치도', '1) 현장 주변현황', '5-1. 가설구조물 등의 설치 및 해체계획', '1) 공통공사', '문단2', '문단3', '문단4', '문서끝']


In [175]:
#db에 문서 탐색 항목들 입력
insert_data(파일이름, 
            현장명,
            현장소재지,
            공사금액,
            보고서_날짜_년,
            보고서_날짜_월,
            시공자,
            설계자,
            공사개요_대상공사,
            공사개요_구조,
            공사개요_개소,
            공사개요_층수지하,
            공사개요_층수지상,
            공사개요_굴착깊이,
            공사개요_최고높이,
            공사개요_연면적,
            기타특수구조물개요,
            주요공법1,
            주요공법2,
            주요공법3,
            주요공법4,
            주요공법5,
            주요공법6,
            주요공법7,
            주요공법8,
            주요공법9,
            주요공법10,
            파일경로,
            정렬된_section_titles,
            정렬된_cursor_positions,
            비고
           )

# 데이터 조회 및 출력
print_data()


(1, '01. [변경] 안전관리계획서 제1편 - 천안 성성5지구 IPARK 신축공사(R4) 정기안전점검 수정 240925.hwp', '현장명', '현장소재지', 1.0, '2025', '09', '시공자', '설계자', '대상공사', '구조', 2, 3, 4, 5.0, 6.7, 8.9, '특수구조물', '공법1', '공법2', '공법3', '공법4', '공법5', '공법6', '공법7', '공법8', '공법9', '공법10', 'C:\\Users\\codek\\Desktop\\건설자료\\01. [변경] 안전관리계획서 제1편 - 천안 성성5지구 IPARK 신축공사(R4) 정기안전점검 수정 240925.hwp', '문서처음', 0, 0, 16, '비고')
(2, '01. [변경] 안전관리계획서 제1편 - 천안 성성5지구 IPARK 신축공사(R4) 정기안전점검 수정 240925.hwp', '현장명', '현장소재지', 1.0, '2025', '09', '시공자', '설계자', '대상공사', '구조', 2, 3, 4, 5.0, 6.7, 8.9, '특수구조물', '공법1', '공법2', '공법3', '공법4', '공법5', '공법6', '공법7', '공법8', '공법9', '공법10', 'C:\\Users\\codek\\Desktop\\건설자료\\01. [변경] 안전관리계획서 제1편 - 천안 성성5지구 IPARK 신축공사(R4) 정기안전점검 수정 240925.hwp', '문단1', 0, 54, 8, '비고')
(3, '01. [변경] 안전관리계획서 제1편 - 천안 성성5지구 IPARK 신축공사(R4) 정기안전점검 수정 240925.hwp', '현장명', '현장소재지', 1.0, '2025', '09', '시공자', '설계자', '대상공사', '구조', 2, 3, 4, 5.0, 6.7, 8.9, '특수구조물', '공법1', '공법2', '공법3', '공법4', '공법5', '공법6', '공법7', '공법8', '공법9', '공법10', 'C:\\Users\\codek\\

In [178]:
#db에서 custom_data 특정 id만 추출
selected_ids = [2, 4,7]  # 사용자가 선택한 ID
custom_data = fetch_custom_data_with_next_positions(selected_ids)
print(custom_data)

[((0, 54, 8), (0, 64, 9), '문단1'), ((0, 68, 12), (0, 72, 12), '2-1. 현장 위치도'), ((0, 87, 9), (0, 99, 0), '1) 공통공사')]


In [56]:
# 데이터 업데이트
update_data(3, 비고="수정된 비고")
# 데이터 조회 및 출력
print_data()

(1, '샘플_원본.hwp', '현장명', '현장소재지', 1.0, '2025', '09', '시공자', '설계자', '대상공사', '구조', 2, 3, 4, 5.0, 6.7, 8.9, '특수구조물', '공법1', '공법2', '공법3', '공법4', '공법5', '공법6', '공법7', '공법8', '공법9', '공법10', 'C:\\Users\\codek\\Documents\\GitHub\\connwnw\\test\\샘플_원본.hwp', '1. 샘플1', 0, 0, 16, '비고')
(2, '샘플_원본.hwp', '현장명', '현장소재지', 1.0, '2025', '09', '시공자', '설계자', '대상공사', '구조', 2, 3, 4, 5.0, 6.7, 8.9, '특수구조물', '공법1', '공법2', '공법3', '공법4', '공법5', '공법6', '공법7', '공법8', '공법9', '공법10', 'C:\\Users\\codek\\Documents\\GitHub\\connwnw\\test\\샘플_원본.hwp', '2. 샘플2', 0, 7, 0, '비고')
(3, '샘플_원본.hwp', '현장명', '현장소재지', 1.0, '2025', '09', '시공자', '설계자', '대상공사', '구조', 2, 3, 4, 5.0, 6.7, 8.9, '특수구조물', '공법1', '공법2', '공법3', '공법4', '공법5', '공법6', '공법7', '공법8', '공법9', '공법10', 'C:\\Users\\codek\\Documents\\GitHub\\connwnw\\test\\샘플_원본.hwp', '3. 샘플3', 0, 10, 0, '수정된 비고')
(4, '샘플_원본.hwp', '현장명', '현장소재지', 1.0, '2025', '09', '시공자', '설계자', '대상공사', '구조', 2, 3, 4, 5.0, 6.7, 8.9, '특수구조물', '공법1', '공법2', '공법3', '공법4', '공법5', '공법6', '공법7', '공법8', '공법

In [ ]:
# 데이터 삭제
delete_data(2)
# 데이터 조회 및 출력
print_data()



In [86]:
custom_data.clear()


In [154]:
#####복붙 TEST 용#####

#내용 삽입 함수
def 복붙_내용삽입(custom_data):
    """
    custom_data: [(섹션시작좌표, 섹션끝좌표, 섹션명), ...] 형태의 리스트
    """
    for section_start, section_end, section_name in custom_data:
        # 섹션 시작 위치로 이동
        hwp.SetPos(*section_start)  
        hwp.MoveNextParaBegin()  # 다음 단락으로 이동
        hwp.Select()  # 섹션 시작 위치에서 선택 시작

        # 섹션 끝 위치로 이동
        if section_end == None:
            print("잘못된 id를 추출하였습니다.(문서끝id입력함)")
        else:
            hwp.SetPos(*section_end)
            hwp.MoveLeft()  # 끝 위치에서 한 글자 왼쪽으로 이동해 선택 범위 조정

        # 복사 작업 수행
        hwp.Copy()

        # 두 번째 HWP 파일의 필드로 이동하여 붙여넣기
        hwp2.MoveToField(section_name)
        hwp2.Paste()  # 복사한 내용 붙여넣기

복붙_내용삽입(custom_data)